In [1]:
import numpy as np
import pandas as pd
import pickle, sys, csv, gzip, glob, os, threading
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem import PandasTools
import sklearn.feature_extraction
from sklearn.feature_selection import VarianceThreshold 
from sklearn.metrics import roc_auc_score
from boruta import boruta_py
from mordred import Calculator, descriptors
from timeit import default_timer as timer
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from collections import defaultdict
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from hyperopt import hp
from hyperopt.pyll.stochastic import sample
from pyspark import SparkContext, SparkConf
from spark_sklearn import GridSearchCV
from sklearn import cross_validation, metrics   
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from hyperopt import STATUS_OK
from timeit import default_timer as timer
import lightgbm as lgb
from hyperopt import tpe
from hyperopt import Trials
from hyperopt import fmin
from sklearn.model_selection import KFold

In [2]:
datasets = ["Homopiperazines","Piperazines","Piperidines","Sulphamides"]
names = ["JAK1 EC50 nM 1027","JAK2 EC50 nM 1024","JAK3 EC50 nM 1026"]
names = names + ['TYK2 EC50 nM 1025']
files = ["/dbfs/FileStore/tables/Homopiperazines_cleaned_Feb_2019.sdf",
         "/dbfs/FileStore/tables/Piperazines_cleaned_Feb_2019.sdf",
         "/dbfs/FileStore/tables/Piperidines_cleaned_Feb_2019.sdf",
         "/dbfs/FileStore/tables/Sulphamides_cleaned_Feb_2019.sdf",
        ]
PARENT_DIR = '/dbfs/FileStore/tables'
PICKLES_DIR = '/dbfs/FileStore/pickles'
MOSES_DIR = '/dbfs/FileStore/moses'
CHEMPROP_DIR = '/dbfs/FileStore/chemprop'
XGB_DIR = '/dbfs/FileStore/XGB-Hopt/'
targets=['JAK1','JAK2','JAK3','TYK2']


In [3]:
def getMorganAsDict(mol):
    d = {}
    d.update(AllChem.GetMorganFingerprint(mol,2).GetNonzeroElements())
    return d
def get_fps(drugs, morgan=False, bit=False):
    fps=[]
    errors = 0
    for moli in range(len(drugs)):
        try:
            mol = drugs[moli]
            if morgan:
                d = getMorganAsDict(mol)
            elif bit:
                d = AllChem.GetMorganFingerprintAsBitVect(mol,2,nBits=2048)
            else:
                d = Chem.RDKFingerprint(mol)
            fps.append(d)
        except:
            errors+=1
    print('Errors in conversion:',errors)
    return fps

# Classification Hyperparameter Optimization

In [5]:
Train_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','train-1460_bin76.csv'))
Val_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','val-182_bin76.csv'))
Test_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','test-183_bin76.csv'))

In [6]:
All_4y = Train_4y.append(Val_4y).append(Test_4y)
all_fps = [Chem.MolFromSmiles(smi) for smi in All_4y['smiles']]
fps = get_fps(all_fps, morgan=True)
v = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
v.fit(fps)
print(len(v.feature_names_))
print(len(v.vocabulary_))

X = v.transform(fps)

X = X.toarray()
Train_X = X[:1460]
Val_X = X[1460:1460+182]
Test_X = X[1460+182:1460+182+183]

Errors in conversion: 0
3689
3689

In [7]:
Train_X = np.concatenate((Train_X,Val_X))
Train_4y = Train_4y.append(Val_4y)

In [8]:

tpe_algorithm = tpe.suggest
bayes_trials = Trials()
iter=0
MAX_EVALS = 100
N_FOLDS = 10
def objective(params, n_folds = N_FOLDS):
    global iter
    iter += 1
    
    subsample = params['boosting_type'].get('subsample', 1.0)
    
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['subsample'] = subsample
    
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    
    cv_results = lgb.cv(params, train_set, num_boost_round = 10000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 50)
    
    run_time = timer() - start
    
    best_score = np.max(cv_results['auc-mean'])
    
    loss = 1 - best_score
    
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)

    logfile = open(out_file, 'a')
    writer = csv.writer(logfile)
    writer.writerow([loss, params, iter, n_estimators, run_time])
    
    return {'loss': loss, 'params': params, 'iteration': iter,
            'estimators': n_estimators, 
            'train_time': run_time, 'status': STATUS_OK}

In [9]:
# Define the search space
space = {
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                                 {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}

out_file = os.path.join(XGB_DIR,'gbm_trials_binary.csv')
logfile = open(out_file, 'w')
writer = csv.writer(logfile)

writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
logfile.close()

In [10]:
global iter
iter=0
lgbmodel = lgb.LGBMClassifier()
train_set = lgb.Dataset(Train_X, label = Train_4y[targets[0]])
# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(13))

0%| | 0/100 [00:00<?, ?it/s, best loss: ?] 1%| | 1/100 [00:05<09:41, 5.87s/it, best loss: 0.08996395691466108] 2%|▏ | 2/100 [00:09<08:19, 5.09s/it, best loss: 0.08996395691466108] 3%|▎ | 3/100 [00:09<05:59, 3.70s/it, best loss: 0.08996395691466108] 4%|▍ | 4/100 [00:13<05:48, 3.63s/it, best loss: 0.08996395691466108] 5%|▌ | 5/100 [00:13<04:19, 2.73s/it, best loss: 0.08996395691466108] 6%|▌ | 6/100 [00:20<06:19, 4.04s/it, best loss: 0.08996395691466108] 7%|▋ | 7/100 [00:30<08:56, 5.77s/it, best loss: 0.08996395691466108] 8%|▊ | 8/100 [00:34<08:09, 5.32s/it, best loss: 0.08996395691466108] 9%|▉ | 9/100 [00:35<05:50, 3.86s/it, best loss: 0.08996395691466108] 10%|█ | 10/100 [00:41<06:56, 4.63s/it, best loss: 0.08996395691466108] 11%|█ | 11/100 [00:45<06:22, 4.30s/it, best loss: 0.08996395691466108] 12%|█▏ | 12/100 [00:50<06:31, 4.45s/it, best loss: 0.08996395691466108] 13%|█▎ | 13/100 [00:51<05:19, 3.67s/it, best loss: 0.08996395691466108] 14%|█▍ | 14/100 [00:52<03:51, 2.70s/it, best loss: 0.08996395691466108] 15%|█▌ | 15/100 [00:54<03:36, 2.55s/it, best loss: 0.08646747099211893] 16%|█▌ | 16/100 [00:56<03:27, 2.47s/it, best loss: 0.08646747099211893] 17%|█▋ | 17/100 [01:03<04:57, 3.58s/it, best loss: 0.08623936260203868] 18%|█▊ | 18/100 [01:11<06:46, 4.95s/it, best loss: 0.08623936260203868] 19%|█▉ | 19/100 [01:11<04:56, 3.66s/it, best loss: 0.08623936260203868] 20%|██ | 20/100 [01:15<04:49, 3.62s/it, best loss: 0.08623936260203868] 21%|██ | 21/100 [01:28<08:24, 6.39s/it, best loss: 0.08409435139364729] 22%|██▏ | 22/100 [01:39<10:10, 7.83s/it, best loss: 0.0825135311367704]  23%|██▎ | 23/100 [01:48<10:34, 8.25s/it, best loss: 0.0825135311367704] 24%|██▍ | 24/100 [02:01<12:21, 9.76s/it, best loss: 0.0825135311367704] 25%|██▌ | 25/100 [02:13<12:49, 10.26s/it, best loss: 0.0825135311367704] 26%|██▌ | 26/100 [02:14<09:17, 7.54s/it, best loss: 0.0825135311367704] 27%|██▋ | 27/100 [02:24<10:08, 8.33s/it, best loss: 0.0825135311367704] 28%|██▊ | 28/100 [02:34<10:23, 8.66s/it, best loss: 0.0825135311367704] 29%|██▉ | 29/100 [02:39<08:56, 7.55s/it, best loss: 0.0825135311367704] 30%|███ | 30/100 [02:40<06:46, 5.81s/it, best loss: 0.0825135311367704] 31%|███ | 31/100 [02:49<07:50, 6.82s/it, best loss: 0.0825135311367704] 32%|███▏ | 32/100 [03:02<09:40, 8.54s/it, best loss: 0.0825135311367704] 33%|███▎ | 33/100 [03:16<11:21, 10.17s/it, best loss: 0.0825135311367704] 34%|███▍ | 34/100 [03:17<08:05, 7.35s/it, best loss: 0.0825135311367704] 35%|███▌ | 35/100 [03:20<06:36, 6.09s/it, best loss: 0.0825135311367704] 36%|███▌ | 36/100 [03:31<08:03, 7.56s/it, best loss: 0.0825135311367704] 37%|███▋ | 37/100 [03:32<05:55, 5.64s/it, best loss: 0.0825135311367704] 38%|███▊ | 38/100 [03:41<06:49, 6.60s/it, best loss: 0.0825135311367704] 39%|███▉ | 39/100 [03:51<07:49, 7.70s/it, best loss: 0.0825135311367704] 40%|████ | 40/100 [03:57<07:14, 7.23s/it, best loss: 0.0825135311367704] 41%|████ | 41/100 [03:58<05:08, 5.23s/it, best loss: 0.0825135311367704] 42%|████▏ | 42/100 [04:05<05:28, 5.66s/it, best loss: 0.0825135311367704] 43%|████▎ | 43/100 [04:16<07:03, 7.42s/it, best loss: 0.0825135311367704] 44%|████▍ | 44/100 [04:19<05:47, 6.21s/it, best loss: 0.0825135311367704] 45%|████▌ | 45/100 [04:20<04:06, 4.48s/it, best loss: 0.0825135311367704] 46%|████▌ | 46/100 [04:35<06:49, 7.58s/it, best loss: 0.0825135311367704] 47%|████▋ | 47/100 [04:40<06:12, 7.03s/it, best loss: 0.0825135311367704] 48%|████▊ | 48/100 [04:53<07:27, 8.60s/it, best loss: 0.0825135311367704] 49%|████▉ | 49/100 [04:56<05:52, 6.91s/it, best loss: 0.0825135311367704] 50%|█████ | 50/100 [05:07<06:53, 8.27s/it, best loss: 0.0825135311367704] 51%|█████ | 51/100 [05:12<05:58, 7.31s/it, best loss: 0.0825135311367704] 52%|█████▏ | 52/100 [05:18<05:24, 6.76s/it, best loss: 0.0825135311367704] 53%|█████▎ | 53/100 [05:19<04:03, 5.18s/it, best loss: 0.0825135311367704] 54%|█████▍ | 54/100 [05:25<04:05, 5.33s/it, best loss: 0.0825135311367704] 55

In [11]:
best

Out[15]: 
{'boosting_type': 0,
 'class_weight': 0,
 'colsample_by_tree': 0.8521732607849484,
 'gdbt_subsample': 0.6680712282914405,
 'learning_rate': 0.18857516048056255,
 'min_child_samples': 25.0,
 'num_leaves': 83.0,
 'reg_alpha': 0.1704362270661588,
 'reg_lambda': 0.4444416565086208,
 'subsample_for_bin': 120000.0}

# Testing the optimal model

In [13]:
from copy import deepcopy
import lightgbm as lgb

lgbmodel = lgb.LGBMClassifier(
                             colsample_by_tree= 0.8521732607849484,
                             gdbt_subsample= 0.6680712282914405,
                             learning_rate= 0.18857516048056255,
                             min_child_samples= 25,
                             num_leaves= 83,
                             reg_alpha= 0.1704362270661588,
                             reg_lambda= 0.4444416565086208,
                             subsample_for_bin= 120000,
                             num_estimators = 100)
predicts = deepcopy(Test_4y)
targets = [name[:4] for name in names]
for name in targets:
  lgbmodel.fit(Train_X,Train_4y[name])

  predicts[name]=lgbmodel.predict(Test_X)
predicts.to_csv(os.path.join(XGB_DIR,'predicts20190501_bin76.csv'),index=None)

/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:

In [14]:
from sklearn.metrics import roc_auc_score
for name in targets:
  print(roc_auc_score(Test_4y[name], predicts[name]))

0.8400863102373531
0.8168082524271845
0.7012624172185431
0.7867857142857144

In [15]:
roc_auc_score(Test_4y[targets], predicts[targets])

Out[17]: 0.801015385085865

# Classifying with external

In [17]:
Train_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','train-8396_bin76.csv')).dropna(subset=['JAK1'])
Val_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','val-182_bin76.csv')).dropna(subset=['JAK1'])
Test_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','test-183_bin76.csv')).dropna(subset=['JAK1'])
names = ['JAK1','JAK2','JAK3','TYK2']

All_4y = Train_4y.append(Val_4y).append(Test_4y)
all_fps = [Chem.MolFromSmiles(smi) for smi in All_4y['smiles']]
fps = get_fps(all_fps, morgan=True)
#get_fps([Chem.MolFromSmiles('Cc1c[nH]c(C(=O)N2CCCN(c3ncnc4[nH]ccc34)CC23CC3)c1')],bit=True)
v = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
v.fit(fps)
print(len(v.feature_names_))
print(len(v.vocabulary_))

X = v.transform(fps)

X = X.toarray()
Train_X = X[:len(Train_4y)]
Val_X = X[len(Train_4y):len(Train_4y)+len(Val_4y)]
Test_X = X[len(Train_4y)+len(Val_4y):len(Train_4y)+len(Val_4y)+len(Test_4y)]

In [18]:
space = {
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                                 {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}

tpe_algorithm = tpe.suggest

bayes_trials = Trials()

out_file = os.path.join(XGB_DIR,'gbm_trials_binary_ext.csv')
logfile = open(out_file, 'w')
writer = csv.writer(logfile)

writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
logfile.close()

global  iter

iter = 0
lgbmodel = lgb.LGBMClassifier()
train_set = lgb.Dataset(Train_X, label = Train_4y[names[0]])
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(13))

0%| | 0/100 [00:00<?, ?it/s, best loss: ?] 1%| | 1/100 [00:19<32:40, 19.80s/it, best loss: 0.038516282305466776] 2%|▏ | 2/100 [00:49<37:21, 22.87s/it, best loss: 0.038516282305466776] 3%|▎ | 3/100 [00:54<28:04, 17.37s/it, best loss: 0.038516282305466776] 4%|▍ | 4/100 [01:07<25:53, 16.18s/it, best loss: 0.038516282305466776] 5%|▌ | 5/100 [01:12<20:09, 12.73s/it, best loss: 0.038516282305466776] 6%|▌ | 6/100 [01:33<23:42, 15.13s/it, best loss: 0.03774286701501084]  7%|▋ | 7/100 [02:40<47:31, 30.66s/it, best loss: 0.03774286701501084] 8%|▊ | 8/100 [03:08<46:07, 30.09s/it, best loss: 0.03774286701501084] 9%|▉ | 9/100 [03:12<33:34, 22.14s/it, best loss: 0.03774286701501084]

In [19]:
res = pd.read_csv('/dbfs/FileStore/XGB-Hopt/gbm_trials_binary_ext.csv')

In [20]:
display(res.sort_values('loss'))

loss,params,iteration,estimators,train_time
0.03518170472841242,"{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.6444568404965568, 'learning_rate': 0.033170339942429246, 'min_child_samples': 25, 'num_leaves': 96, 'reg_alpha': 0.2800473375282556, 'reg_lambda': 0.8509243004571374, 'subsample_for_bin': 100000, 'subsample': 0.9157298770833132}",88,269,23.216553182999636
0.035550194663581884,"{'boosting_type': 'gbdt', 'class_weight': 'balanced', 'colsample_bytree': 0.6810810724555437, 'learning_rate': 0.05189020144801292, 'min_child_samples': 20, 'num_leaves': 96, 'reg_alpha': 0.335050267108593, 'reg_lambda': 0.781863798041295, 'subsample_for_bin': 160000, 'subsample': 0.8332272098874306}",96,135,15.148115986998164
0.036039255118631486,"{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.7282933711143356, 'learning_rate': 0.022642460325559895, 'min_child_samples': 20, 'num_leaves': 104, 'reg_alpha': 0.5059671136057231, 'reg_lambda': 0.7640988911487885, 'subsample_for_bin': 140000, 'subsample': 0.8941408039269114}",83,414,32.23734414300088
0.03632876310502442,"{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.6758002656096727, 'learning_rate': 0.04738208343798852, 'min_child_samples': 30, 'num_leaves': 101, 'reg_alpha': 0.3780343611879261, 'reg_lambda': 0.7820309052243114, 'subsample_for_bin': 160000, 'subsample': 0.7658184241189976}",95,177,15.97161795500142
0.03660333356015999,"{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.9823107708721153, 'learning_rate': 0.023306191871832152, 'min_child_samples': 35, 'num_leaves': 99, 'reg_alpha': 0.43395329328238774, 'reg_lambda': 0.6592580530881157, 'subsample_for_bin': 140000, 'subsample': 0.8598521562626836}",75,437,33.24361484200017
0.036643012822757386,"{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.6025123650190819, 'learning_rate': 0.10273137655295785, 'min_child_samples': 25, 'num_leaves': 112, 'reg_alpha': 0.020282432495463076, 'reg_lambda': 0.5856719744939707, 'subsample_for_bin': 80000, 'subsample': 0.5017532056524744}",23,77,13.364152792999446
0.036679600194762685,"{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.9949941124663952, 'learning_rate': 0.03692724017845553, 'min_child_samples': 35, 'num_leaves': 87, 'reg_alpha': 0.6829828429191762, 'reg_lambda': 0.4539467910080274, 'subsample_for_bin': 180000, 'subsample': 0.57747789786517}",69,307,23.024926374000646
0.03669778817510405,"{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.998042500503178, 'learning_rate': 0.010921685572671265, 'min_child_samples': 35, 'num_leaves': 117, 'reg_alpha': 0.6198403059803564, 'reg_lambda': 0.3566920128198412, 'subsample_for_bin': 200000, 'subsample': 0.5895270869249793}",46,961,61.76514977200168
0.036712190571451364,"{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.9994656071585971, 'learning_rate': 0.05703159053156296, 'min_child_samples': 30, 'num_leaves': 90, 'reg_alpha': 0.702083864804418, 'reg_lambda': 0.2937568732451366, 'subsample_for_bin': 200000, 'subsample': 0.520575442044964}",48,170,15.253554793000147
0.03672822479894455,"{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.9926780687485904, 'learning_rate': 0.057003567638112394, 'min_child_samples': 20, 'num_leaves': 89, 'reg_alpha': 0.7232282823597782, 'reg_lambda': 0.2770302575991247, 'subsample_for_bin': 200000, 'subsample': 0.5124735224173583}",66,165,15.646668150999176


In [21]:


lgbmodel = lgb.LGBMClassifier(
                             colsample_bytree=0.6444568404965568,
                             gdbt_subsample=0.9157298770833132,
                             learning_rate=0.033170339942429246,
                             min_child_samples=25,
                             num_leaves=96,
                             reg_alpha=0.2800473375282556,
                             reg_lambda=0.8509243004571374,
                             subsample_for_bin=100000, subsample = 0.9157298770833132,n_estimators=100)
predicts = Test_4y
for name in names:
  Train_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','train-8396_bin76.csv')).dropna(subset=[name])
  names = ['JAK1','JAK2','JAK3','TYK2']
  All_4y = Train_4y.append(Val_4y).append(Test_4y)
  all_fps = [Chem.MolFromSmiles(smi) for smi in All_4y['smiles']]
  fps = get_fps(all_fps, morgan=True)
  v = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
  v.fit(fps)
  print(len(v.feature_names_))
  print(len(v.vocabulary_))

  X = v.transform(fps)

  X = X.toarray()
  Train_X = X[:len(Train_4y)]
  Val_X = X[len(Train_4y):len(Train_4y)+len(Val_4y)]
  Test_X = X[len(Train_4y)+len(Val_4y):len(Train_4y)+len(Val_4y)+len(Test_4y)]
  
  lgbmodel.fit(Train_X,Train_4y[name])

  predicts[name]=lgbmodel.predict(Test_X)
#rmse = np.sqrt(mean_squared_error(Test_4y[names[0]], y_pred))
predicts.to_csv(os.path.join(XGB_DIR,'predicts20190501_binary_ext.csv'),index=None)
Train_X = np.concatenate((Train_X,Val_X))
Train_4y = Train_4y.append(Val_4y)

Errors in conversion: 0
10256
10256
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
Errors in conversion: 0
12697
12697
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
Errors in conversion: 0
10262
10262
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
Errors in conversion: 0
7312
7312
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:

In [22]:
from sklearn.metrics import roc_auc_score
Test_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','test-183_bin76.csv')).dropna(subset=['JAK1'])

for name in names:
  print(roc_auc_score(Test_4y[name], predicts[name]))

0.8110165427954926
0.8341626213592234
0.6264486754966887
0.6221428571428571

# unused material

In [24]:
full_df = pd.concat([full_internal, full_descriptors], axis=1)
m_cols = full_descriptors.columns
temp = full_df.dropna(subset=names)
Xs_all = temp[m_cols]
ys_all = temp[names].apply(pd.to_numeric,errors='coerce').apply(lambda x: x*1e-9).apply(np.log10)
ys_all = -ys_all

corr_matrix = Xs_all.corr().abs()
#upper triangle
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [25]:
X_train, X_test, y_train, y_test = train_test_split(Xs_all, ys_all[names[0]], test_size=0.2, random_state=7)
gbr = XGBRegressor(
  learning_rate =0.1,
  n_estimators=150,
  max_depth=6,
  min_child_weight=1,
  gamma=0,
  subsample=0.8,
  colsample_bytree=0.8,
  nthread=4,
  scale_pos_weight=1,
  seed=27)


def filter_x_by_corr(thresh):
  to_drop = [column for column in upper.columns if any(upper[column] > thresh)]
  X_train.f = X_train.drop(full_descriptors[to_drop], axis=1)
  X_test.f = X_test.drop(full_descriptors[to_drop], axis=1)
  return X_train.f, X_test.f

thresholds=[0.2,0.3,0.4,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.9]
results = defaultdict(list)
for thresh in thresholds:
  X_train.f, X_test.f = filter_x_by_corr(thresh)
  gbr.fit(X_train.f, y_train)
  y_pred = gbr.predict(X_test.f)
  #predictions = [round(value) for value in y_pred]
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  var_score = explained_variance_score(y_test, y_pred)
  results['Threshold'].append(thresh)
  results['N_descriptors'].append(X_train.f.shape[1])
  results['RMSE'].append(round(rmse, 2))
  results['Explained_var'].append(round(var_score, 2))
display(pd.DataFrame.from_dict(results))

In [26]:
param_test = {
 'max_depth':range(3,10),
 'min_child_weight':range(1,8),
 'gamma':[i/10.0 for i in range(0,8)],
 'subsample':[i/10.0 for i in range(4,12)],
 'colsample_bytree':[i/10.0 for i in range(4,12)],
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

# trying out

In [28]:
Train_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','train-1460.csv'))
Val_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','val-182.csv'))
Test_4y = pd.read_csv(os.path.join(CHEMPROP_DIR,'JAK','test-183.csv'))

In [29]:
print(len(Train_4y))
print(len(Val_4y))
print(len(Test_4y))

1460
182
183

In [30]:
All_4y = Train_4y.append(Val_4y).append(Test_4y)

In [31]:
display(Train_4y.head())

smiles,JAK1 EC50 nM 1027,JAK2 EC50 nM 1024,JAK3 EC50 nM 1026,TYK2 EC50 nM 1025
Cc1c[nH]c(C(=O)N2CCCN(c3ncnc4[nH]ccc34)CC23CC3)c1,7.856985199745905,7.946921556516581,6.886056647693162,6.82102305270683
O=S(=O)(N(CCN1CCOCC1)C1CCC1)N1CCN(c2ncnc3[nH]ccc23)CC12CC2,7.0695604052332985,7.357535479757877,6.33161408331,6.157390760389438
CN(C[C@@H]1CCCN1S(C)(=O)=O)S(=O)(=O)N1CCN(c2ncnc3[nH]ccc23)CC12CC2,7.732828271596986,7.832682665251823,6.966576244513051,6.376750709602098
CC(=O)N1CC[C@H]1COC(=O)N1C2CCC1CN(c1ncnc3[nH]ccc13)C2,6.7544873321858505,6.665546248849069,6.244887733604928,5.379863945026242
N#Cc1ccc(CC(=O)N2CCN(c3ncnc4[nH]ccc34)C3(CC3)C2)cc1,6.5044556624535526,6.1487416512809245,5.782516055786092,5.235077015350113


In [32]:
all_fps = [Chem.MolFromSmiles(smi) for smi in All_4y['smiles']]
fps = get_fps(all_fps, morgan=True)
#get_fps([Chem.MolFromSmiles('Cc1c[nH]c(C(=O)N2CCCN(c3ncnc4[nH]ccc34)CC23CC3)c1')],bit=True)
v = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
v.fit(fps)
print(len(v.feature_names_))
print(len(v.vocabulary_))

X = v.transform(fps)

X = X.toarray()

Errors in conversion: 0
3689
3689

In [33]:
Train_X = X[:1460]
Val_X = X[1460:1460+182]
Test_X = X[1460+182:1460+182+183]

In [34]:
# Model with default hyperparameters
lgbmodel = lgb.LGBMRegressor()
lgbmodel

Out[13]: 
LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
 importance_type='split', learning_rate=0.1, max_depth=-1,
 min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
 n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
 random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
 subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [35]:
xgbmodel = XGBRegressor()
xgbmodel

Out[14]: 
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
 colsample_bytree=1, gamma=0, importance_type='gain',
 learning_rate=0.1, max_delta_step=0, max_depth=3,
 min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
 nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
 reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
 subsample=1)

In [36]:
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer
iter=0
def objective(params, n_folds = N_FOLDS):
    global iter
    iter += 1
    
    subsample = params['boosting_type'].get('subsample', 1.0)
    
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['subsample'] = subsample
    
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    
    cv_results = lgb.cv(params, train_set, num_boost_round = 10000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = 'rmse', seed = 13,stratified=False)
    
    run_time = timer() - start
    
    loss = np.min(cv_results['rmse-mean'])
    print('best_score:',loss)
    
    n_estimators = int(np.argmin(cv_results['rmse-mean']) + 1)

    logfile = open(out_file, 'a')
    writer = csv.writer(logfile)
    writer.writerow([loss, params, iter, n_estimators, run_time])
    
    return {'loss': loss, 'params': params, 'iteration': iter,
            'estimators': n_estimators, 
            'train_time': run_time, 'status': STATUS_OK}

In [37]:
space = {
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                                 {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}

In [38]:

out_file = os.path.join(XGB_DIR,'gbm_trials.csv')
logfile = open(out_file, 'w')
writer = csv.writer(logfile)

writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
logfile.close()

In [39]:
global iter
iter=0
lgbmodel = lgb.LGBMRegressor()
train_set = lgb.Dataset(Train_X, label = Train_4y[names[0]])
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(13))

0%| | 0/100 [00:00<?, ?it/s, best loss: ?] best_score:
 0%| | 0/100 [00:07<?, ?it/s, best loss: ?] 0.5502175798729334
 0%| | 0/100 [00:07<?, ?it/s, best loss: ?] 1%| | 1/100 [00:07<12:51, 7.79s/it, best loss: 0.5502175798729334] best_score:
 1%| | 1/100 [00:27<12:51, 7.79s/it, best loss: 0.5502175798729334] 0.7647355929918908
 1%| | 1/100 [00:27<12:51, 7.79s/it, best loss: 0.5502175798729334] 2%|▏ | 2/100 [00:28<18:50, 11.53s/it, best loss: 0.5502175798729334] best_score:
 2%|▏ | 2/100 [00:28<18:50, 11.53s/it, best loss: 0.5502175798729334] 0.9203129687560923
 2%|▏ | 2/100 [00:28<18:50, 11.53s/it, best loss: 0.5502175798729334] 3%|▎ | 3/100 [00:28<13:13, 8.18s/it, best loss: 0.5502175798729334] best_score:
 3%|▎ | 3/100 [00:32<13:13, 8.18s/it, best loss: 0.5502175798729334] 0.5543020993080805
 3%|▎ | 3/100 [00:32<13:13, 8.18s/it, best loss: 0.5502175798729334] 4%|▍ | 4/100 [00:32<10:57, 6.85s/it, best loss: 0.5502175798729334] best_score:
 4%|▍ | 4/100 [00:32<10:57, 6.85s/it, best loss: 0.5502175798729334] 0.7158925575522768
 4%|▍ | 4/100 [00:32<10:57, 6.85s/it, best loss: 0.5502175798729334] 5%|▌ | 5/100 [00:32<07:50, 4.96s/it, best loss: 0.5502175798729334] best_score:
 5%|▌ | 5/100 [00:44<07:50, 4.96s/it, best loss: 0.5502175798729334] 0.5602758413987451
 5%|▌ | 5/100 [00:44<07:50, 4.96s/it, best loss: 0.5502175798729334] 6%|▌ | 6/100 [00:44<10:52, 6.94s/it, best loss: 0.5502175798729334] best_score:
 6%|▌ | 6/100 [01:07<10:52, 6.94s/it, best loss: 0.5502175798729334] 0.5619718060026441
 6%|▌ | 6/100 [01:07<10:52, 6.94s/it, best loss: 0.5502175798729334] 7%|▋ | 7/100 [01:07<18:12, 11.75s/it, best loss: 0.5502175798729334] best_score:
 7%|▋ | 7/100 [01:20<18:12, 11.75s/it, best loss: 0.5502175798729334] 0.6790442365117748
 7%|▋ | 7/100 [01:20<18:12, 11.75s/it, best loss: 0.5502175798729334] 8%|▊ | 8/100 [01:20<18:32, 12.09s/it, best loss: 0.5502175798729334] best_score:
 8%|▊ | 8/100 [01:20<18:32, 12.09s/it, best loss: 0.5502175798729334] 0.8557748901756701
 8%|▊ | 8/100 [01:20<18:32, 12.09s/it, best loss: 0.5502175798729334] 9%|▉ | 9/100 [01:20<12:59, 8.56s/it, best loss: 0.5502175798729334] best_score:
 9%|▉ | 9/100 [01:42<12:59, 8.56s/it, best loss: 0.5502175798729334] 0.5951413896301494
 9%|▉ | 9/100 [01:42<12:59, 8.56s/it, best loss: 0.5502175798729334] 10%|█ | 10/100 [01:42<18:48, 12.54s/it, best loss: 0.5502175798729334] best_score:
 10%|█ | 10/100 [01:58<18:48, 12.54s/it, best loss: 0.5502175798729334] 0.7987255422215165
 10%|█ | 10/100 [01:58<18:48, 12.54s/it, best loss: 0.5502175798729334] 11%|█ | 11/100 [01:58<20:15, 13.66s/it, best loss: 0.5502175798729334] best_score:
 11%|█ | 11/100 [02:21<20:15, 13.66s/it, best loss: 0.5502175798729334] 0.8838805640230598
 11%|█ | 11/100 [02:21<20:15, 13.66s/it, best loss: 0.5502175798729334] 12%|█▏ | 12/100 [02:22<24:24, 16.65s/it, best loss: 0.5502175798729334] best_score:
 12%|█▏ | 12/100 [02:32<24:24, 16.65s/it, best loss: 0.5502175798729334] 0.8707299226088789
 12%|█▏ | 12/100 [02:32<24:24, 16.65s/it, best loss: 0.5502175798729334] 13%|█▎ | 13/100 [02:32<21:33, 14.86s/it, best loss: 0.5502175798729334] best_score:
 13%|█▎ | 13/100 [02:33<21:33, 14.86s/it, best loss: 0.5502175798729334] 0.9188147086835656
 13%|█▎ | 13/100 [02:33<21:33, 14.86s/it, best loss: 0.5502175798729334] 14%|█▍ | 14/100 [02:33<15:07, 10.55s/it, best loss: 0.5502175798729334] best_score:
 14%|█▍ | 14/100 [02:36<15:07, 10.55s/it, best loss: 0.5502175798729334] 0.5444591314072215
 14%|█▍ | 14/100 [02:36<15:07, 10.55s/it, best loss: 0.5502175798729334] 15%|█▌ | 15/100 [02:36<11:40, 8.24s/it, best loss: 0.5444591314072215] best_score:
 15%|█▌ | 15/100 [02:59<11:40, 8.24s/it, best loss: 0.5444591314072215] 0.8842015351135079
 15%|█▌ | 15/100 [02:59<11:40, 8.24s/it, best loss: 0.5444591314072215] 16%|█▌ | 16/100 [03:00<18:06, 12.93s/it, best loss: 0.5444591314072215] best_score:
 16%|█▌ | 16/10

In [40]:
best

Out[26]: 
{'boosting_type': 0,
 'class_weight': 0,
 'colsample_by_tree': 0.7593855006084105,
 'gdbt_subsample': 0.7318441942731282,
 'learning_rate': 0.013309721427152973,
 'min_child_samples': 20.0,
 'num_leaves': 123.0,
 'reg_alpha': 0.22991023483702192,
 'reg_lambda': 0.39725196762110565,
 'subsample_for_bin': 180000.0}

In [41]:
import lightgbm as lgb
from sklearn import cross_validation

clf = lgb.sklearn.LGBMRegressor(
                             colsample_bytree=0.7593855006084105,
                             gdbt_subsample=0.7318441942731282,
                             learning_rate=0.013309721427152973,
                             min_child_samples=20,
                             num_leaves=123,
                             reg_alpha=0.22991023483702192,
                             reg_lambda=0.39725196762110565,
                             subsample_for_bin=180000, metrics='rmse',num_boost_round = 10000)



Train_X = X[:1460+182]
Train_4y = All_4y[:1460+182]

Test_X = X[1460+182:1460+182+183]

predicts = Test_4y
for name in names:
  clf.fit(Train_X,Train_4y[name])
  predicts[name]=clf.predict(Test_X)
predicts.to_csv(os.path.join(XGB_DIR,'predicts20190514-Test.csv'),index=None)

/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

In [42]:
import lightgbm as lgb
from sklearn import cross_validation

clf = lgb.sklearn.LGBMRegressor(
                             colsample_bytree=0.7593855006084105,
                             gdbt_subsample=0.7318441942731282,
                             learning_rate=0.013309721427152973,
                             min_child_samples=20,
                             num_leaves=123,
                             reg_alpha=0.22991023483702192,
                             reg_lambda=0.39725196762110565,
                             subsample_for_bin=180000, metrics='rmse',num_boost_round = 10000)

predicts = All_4y
for name in names:
  scores = cross_validation.cross_val_score(
    clf, X, All_4y[name], cv=10, scoring='neg_mean_squared_error')
  print("%s Accuracy: %0.5f (+/- %0.5f)" % (
    clf.__class__.__name__, scores.mean(), scores.std() * 2))
  clf.fit(X,All_4y[name])
  predicts[name]=clf.predict(X)
predicts.to_csv(os.path.join(XGB_DIR,'predicts20190513-All.csv'),index=None)

/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
LGBMRegressor Accuracy: -0.23049 (+/- 0.09103)
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
 warnings.

In [43]:
import seaborn as sns; sns.set(color_codes=True)
All_4y = Train_4y.append(Val_4y).append(Test_4y)
diff_cp = All_4y.filter(names).subtract(predicts.filter(names)).abs()
diff_cp.columns=diff_cp.columns+['diff']
plt.close()
f, axes = plt.subplots(1, 4, sharex = True, sharey=True, figsize=(16,4))
for i in range(4):
  sns.kdeplot(All_4y[names[i]], diff_cp[diff_cp.columns[i]], n_levels=10, cmap="Purples_d",ax=axes[i],clip=((0,10),(0,1)))#
plt.tight_layout()
display(plt.show())

In [44]:
lgbmodel = lgb.LGBMRegressor(
                             colsample_bytree=0.7593855006084105,
                             gdbt_subsample=0.7318441942731282,
                             learning_rate=0.013309721427152973,
                             min_child_samples=20,
                             num_leaves=123,
                             reg_alpha=0.22991023483702192,
                             reg_lambda=0.39725196762110565,
                             subsample_for_bin=180000)
predicts = Test_4y
for name in names:
  lgbmodel.fit(Train_X,Train_4y[name])

  predicts[name]=lgbmodel.predict(Test_X)


In [45]:
predicts.to_csv(os.path.join(XGB_DIR,'predicts20190429.csv'),index=None)

# 4-fold domain CV

In [47]:
%run /Users/vxjdk@leo-pharma.com/mol_utils

In [48]:
domains = ['Homopiperazines','Piperazines','Piperidines','Sulphamides']
frames = []
for j, file in enumerate(files):
  frame = PandasTools.LoadSDF(file,
                              smilesName='SMILES',molColName='Molecule', includeFingerprints=False)
  frame.replace('newline',np.nan, inplace=True)
  
  frame.dropna(subset=names,inplace=True)
  frame['domain'] = [domains[j] for i in range(len(frame))]
  frames.append(frame)

In [49]:
all_bin = pd.concat(frames)
all_bin.columns

/local_disk0/tmp/1557118074189-0/PythonShell.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

 from __future__ import absolute_import
Out[31]: 
Index(['AlogP', 'Canonical Smiles', 'HBA', 'HBD', 'HLM Cl_app mL/min/kg 1065',
 'ID', 'JAK1 EC50 nM 1027', 'JAK2 EC50 nM 1024', 'JAK3 EC50 nM 1026',
 'Leonumber', 'LogD pH7.4 1192', 'Mol Psa', 'Molecule', 'Mw g/mol',
 'SMILES', 'STAT6 EC50 nM 1075', 'STAT6 Emax % 1075',
 'TYK2 EC50 nM 1025', 'domain', 'ope 1024', 'ope 1025', 'ope 1026',
 'ope 1027', 'ope 1065', 'ope 1075', 'ope 1192'],
 dtype='object')

In [50]:
for name in names:
  all_bin[name] = all_bin[name].astype(float)
  all_bin[name] = all_bin[name].apply(lambda x: 1 if x < 10 else 0)

In [51]:
all_bin = all_bin.filter(['Canonical Smiles','domain']+names)
print(all_bin.columns)

Index(['Canonical Smiles', 'domain', 'JAK1 EC50 nM 1027', 'JAK2 EC50 nM 1024',
 'JAK3 EC50 nM 1026', 'TYK2 EC50 nM 1025'],
 dtype='object')

In [52]:
all_fps = [Chem.MolFromSmiles(smi) for smi in all_bin['Canonical Smiles']]
fps = get_fps(all_fps, morgan=True)
v = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
v.fit(fps)
print(len(v.feature_names_))
print(len(v.vocabulary_))

X = v.transform(fps)

X = X.toarray()
print(X.shape)

Errors in conversion: 0
3689
3689
(1825, 3689)

In [53]:
domains

Out[145]: []

In [54]:
from sklearn.model_selection import cross_val_score
cross_val_score(lgbmodel, Train_X, Train_4y[names[0]])

/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
Out[51]: array([0.85948905, 0.80438757, 0.85009141])

In [55]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample
# Define the search space
space = {
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                                 {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    #'n_estimators': hp.quniform('num_trees', 20, 1500, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}

In [56]:
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer
from hyperopt import tpe

# optimization algorithm
tpe_algorithm = tpe.suggest

from hyperopt import Trials

# Keep track of results
bayes_trials = Trials()

# File to save first results
out_file = os.path.join(XGB_DIR,'gbm_testing_4cv_binary.csv')
logfile = open(out_file, 'w')
writer = csv.writer(logfile)

# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
logfile.close()
from hyperopt import fmin

In [57]:
#lgbmodel = lgb.LGBMRegressor()
#train_set = lgb.Dataset(Train_X, label = Train_4y[names[0]])
# Run optimization
global  iter

iter = 0

best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(13))

In [58]:
lgbmodel = lgb.LGBMClassifier(boosting_type='goss',
                             n_estimators=36,
                             class_weight=None,
                             colsample_bytree=0.8372995638410616,
                             learning_rate=0.1958296478537693,
                             min_child_samples=185,
                             num_leaves=46,
                             reg_alpha=0.6012356293352163,
                             reg_lambda=0.4495316419073581,
                             subsample_for_bin=120000)
predicts = all_bin.filter(names)
for name in names:
  for domain in domains:
    rest = domains[:]
    rest.remove(domain)

    lgbmodel.fit(X[all_bin.domain.isin(rest)],all_bin[all_bin.domain.isin(rest)][name].tolist())
    preds = np.round(lgbmodel.predict(X[all_bin.domain==domain]))
    print(preds)
    predicts.loc[all_bin.domain==domain, name] = preds

#predicts.to_csv(os.path.join(XGB_DIR,'predicts20190501_binary.csv'),index=None)

/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
/databricks/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [59]:
display(predicts)

JAK1 EC50 nM 1027,JAK2 EC50 nM 1024,JAK3 EC50 nM 1026,TYK2 EC50 nM 1025
0,0,0,0
0,0,0,0
0,0,0,0
0,0,0,0
0,0,0,0
0,0,0,0
0,0,0,0
0,0,0,0
0,0,0,0
0,0,0,0


In [60]:
train_data = lgb.Dataset(train[0],label=train[1])
validation_data = lgb.Dataset(test[0],label=test[1], reference=train_data)
lgbmodel = lgb.train(params=param, train_set = train_data, num_boost_round=10000, valid_sets=[validation_data],early_stopping_rounds =100)

/databricks/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
 warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[1]	valid_0's auc: 0.823306
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.831843
[3]	valid_0's auc: 0.832656
[4]	valid_0's auc: 0.836856
[5]	valid_0's auc: 0.843496
[6]	valid_0's auc: 0.85481
[7]	valid_0's auc: 0.857385
[8]	valid_0's auc: 0.86416
[9]	valid_0's auc: 0.86416
[10]	valid_0's auc: 0.867005
[11]	valid_0's auc: 0.868428
[12]	valid_0's auc: 0.86897
[13]	valid_0's auc: 0.871748
[14]	valid_0's auc: 0.872696
[15]	valid_0's auc: 0.874322
[16]	valid_0's auc: 0.872967
[17]	valid_0's auc: 0.875271
[18]	valid_0's auc: 0.875
[19]	valid_0's auc: 0.877913
[20]	valid_0's auc: 0.878794
[21]	valid_0's auc: 0.87893
[22]	valid_0's auc: 0.880149
[23]	valid_0's auc: 0.880556
[24]	valid_0's auc: 0.879065
[25]	valid_0's auc: 0.881707
[26]	valid_0's auc: 0.880488
[27]	valid_0's auc: 0.88252
[28]	valid_0's auc: 0.883604
[29]	valid_0's auc: 0.884892
[30]	valid_0's auc: 0.884892
[31]	valid_0's auc: 0.885434
[32]	valid_0's auc: 0.88706
[33]	valid_0's auc: 0.887737
[34]	valid_0's auc: 0.889228
[35]	valid_0's auc: 0.890854
[36]	valid_0's auc: 0.891192
[37]	valid_0's auc: 0.889566
[38]	valid_0's auc: 0.890921
[39]	valid_0's auc: 0.892683
[40]	valid_0's auc: 0.89458
[41]	valid_0's auc: 0.895122
[42]	valid_0's auc: 0.896341
[43]	valid_0's auc: 0.89607
[44]	valid_0's auc: 0.89607
[45]	valid_0's auc: 0.897019
[46]	valid_0's auc: 0.898238
[47]	valid_0's auc: 0.898374
[48]	valid_0's auc: 0.900271
[49]	valid_0's auc: 0.900813
[50]	valid_0's auc: 0.900949
[51]	valid_0's auc: 0.900678
[52]	valid_0's auc: 0.901491
[53]	valid_0's auc: 0.900949
[54]	valid_0's auc: 0.900813
[55]	valid_0's auc: 0.900949
[56]	valid_0's auc: 0.90122
[57]	valid_0's auc: 0.901355
[58]	valid_0's auc: 0.901762
[59]	valid_0's auc: 0.901897
[60]	valid_0's auc: 0.901491
[61]	valid_0's auc: 0.902033
[62]	valid_0's auc: 0.902981
[63]	valid_0's auc: 0.901762
[64]	valid_0's auc: 0.902033
[65]	valid_0's auc: 0.901897
[66]	valid_0's auc: 0.900949
[67]	valid_0's auc: 0.901897
[68]	valid_0's auc: 0.901762
[69]	valid_0's auc: 0.900678
[70]	valid_0's auc: 0.901084
[71]	valid_0's auc: 0.902168
[72]	valid_0's auc: 0.903388
[73]	valid_0's auc: 0.903659
[74]	valid_0's auc: 0.903659
[75]	valid_0's auc: 0.903794
[76]	valid_0's auc: 0.903523
[77]	valid_0's auc: 0.904065
[78]	valid_0's auc: 0.90393
[79]	valid_0's auc: 0.903659
[80]	valid_0's auc: 0.902981
[81]	valid_0's auc: 0.90271
[82]	valid_0's auc: 0.902033
[83]	valid_0's auc: 0.902033
[84]	valid_0's auc: 0.902439
[85]	valid_0's auc: 0.902033
[86]	valid_0's auc: 0.901897
[87]	valid_0's auc: 0.901626
[88]	valid_0's auc: 0.902168
[89]	valid_0's auc: 0.902575
[90]	valid_0's auc: 0.902304
[91]	valid_0's auc: 0.902846
[92]	valid_0's auc: 0.903252
[93]	valid_0's auc: 0.902981
[94]	valid_0's auc: 0.90271
[95]	valid_0's auc: 0.903659
[96]	valid_0's auc: 0.903388
[97]	valid_0's auc: 0.901897
[98]	valid_0's auc: 0.902304
[99]	valid_0's auc: 0.902575
[100]	valid_0's auc: 0.902439
Did not meet early stopping. Best iteration is:
[77]	valid_0's auc: 0.904065

In [61]:
lgbmodel.best_iteration

Out[142]: 77

In [62]:
lgbmodel.params

Out[102]: {'num_leaves': 31, 'objective': 'binary', 'metric': 'auc'}

In [63]:
predicts.to_csv(os.path.join(XGB_DIR,'predicts20190429.csv'),index=None)